In [1]:
# design goals:
#
# - understandability
# - modularity
# - configurability

In [ ]:
# config

WORKSPACE_DIR="workspaces/default"
# replace with pathlib

# mkdir etc

In [ ]:
# conversion

CONVERSION_OUTPUT_DIR = f"{WORKSPACE_DIR}/conversion"
# replace with pathlib

In [ ]:
# chunking

In [ ]:
# authoring

In [ ]:
# sdg